In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

# Loading Data

In [ ]:
data_file = '../input/stackindex/MLTollsStackOverflow.csv'

In [ ]:
def data(data_file):
    data = pd.read_csv(data_file)
    return data

In [ ]:
def load_data():
    data_df = pd.DataFrame()
    data_df = data(data_file)

    timeseries  = data_df['time-series']
    time  = data_df['month']

    series = np.array(timeseries)
    time = np.array(time)
    
    split_time = 112
    
    x_train = series[:split_time]
    x_valid = series[split_time:]

    time_train = time[:split_time]
    time_valid = time[split_time:]

    return data_df, series, time, x_train, x_valid, time_train, time_valid

In [ ]:
data_df, series, time, x_train, x_valid, time_train, time_valid = load_data()

In [ ]:
window_size = 8
batch_size = 32
shuffle_buffer_size = 1000

# Windowed Datasets

In [ ]:

@tf.autograph.experimental.do_not_convert
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w:w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

@tf.autograph.experimental.do_not_convert
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
print(train_set)
print(x_train.shape)

# Visualzing the Data

In [ ]:
plt.figure(figsize=(4, 4));
plt.plot(series);
plt.grid(True)

# Building Model

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

def build_model():
    
    input_layer = tf.keras.Input(shape=(None, 1))
    
    x = tf.keras.layers.Conv1D(filters=32, kernel_size=5, padding='causal', activation='relu')(input_layer)
    x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(32, return_sequences=True)(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(10, activation='relu')(x)
    
    output_layer = tf.keras.layers.Dense(1)(x)

    model = Model(inputs=input_layer, outputs=output_layer)

    return model

model = build_model()

model.summary()

# def piecewise_constant_fn(epoch):
#     if epoch < 110:
#         return 1e-4
#     elif epoch < 120:
#         return 1e-3  
#     else:
#         return 5e-3

# def piecewise_constant(boundaries, values):
#     boundaries = np.array([0] + boundaries)
#     values = np.array(values)
#     def piecewise_constant_fn(epoch):
#         return values[np.argmax(boundaries > epoch) - 1]
#     return piecewise_constant_fn

# piecewise_constant_fn = piecewise_constant([110, 120, ], [1e-4, 1e-3, 5e-3])
# lr_scheduler_pc = keras.callbacks.LearningRateScheduler(piecewise_constant_fn)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=['mae'])

history = model.fit(train_set, epochs=250)#, callbacks=[lr_scheduler_pc]) 

In [ ]:
pd.DataFrame(history.history).plot();

# Plotting Func

In [ ]:
def plot_series(time, series, format='-', start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel('Value')
    plt.grid(True)

# Forecasting

In [ ]:

split_time = 112

rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

In [ ]:
plt.figure(figsize=(5, 5))
plot_series(time_train, x_train)
plot_series(time_valid, x_valid)
#plot_series(time_valid, rnn_forecast)